In [4]:
import pickle
import torch
import sys
sys.path.append('../mag_num/')
from model import *
import numpy as np
from MagNumEvaluator import MagNumEvaluator
from tqdm import tqdm_notebook
from src.model import *
from src.dataset import *

In [115]:
# path = '../probing/model_infre_low/MLP3Diff/Token/12-30-18-11-19-MLP3/model-1686.52938125-0.08-50-1.0.pt'
path = '../probing/model_infre_low/MLP3Diff/p-200/12-30-18-56-42-MLP3/model-20413.11525-0.08-50-1.0.pt'

In [116]:
def load_embed(path):
    with open(path, 'rb') as f:
        embed = pickle.load(f)
        return embed

def dump_embed(path, obj):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

In [117]:
# model = MLP3Diff(50, torch.from_numpy(load_embed('embed_infre_low/Token')).float())
model = MLP3Diff(50, torch.from_numpy(np.random.rand(2342,300)).float())
model.eval()

MLP3Diff(
  (embedding): Embedding(2342, 300)
  (FC1): Linear(in_features=600, out_features=50, bias=True)
  (FC2): Linear(in_features=50, out_features=50, bias=True)
  (FC3): Linear(in_features=50, out_features=1, bias=True)
  (RELU): ReLU()
)

In [118]:
model.load_state_dict(torch.load(path, map_location='cpu'))

In [119]:
test_dataset = pickle.load(open('./data/{}.diff.{}pkl'.format('test', 'lownolog.'),'rb'))
test_dataset = ProbingListMaxDataset(test_dataset)
val_dataset = pickle.load(open('./data/{}.diff.{}pkl'.format('dev', 'lownolog.'),'rb'))
val_dataset = ProbingListMaxDataset(val_dataset)

In [120]:
def val(model, dataset):

    dataLoader = DataLoader(dataset, batch_size=10000, shuffle=True)

    criterion = torch.nn.MSELoss()

    val_loss = 0

    model.eval()
    with torch.no_grad():
        for batch in dataLoader:
            x = torch.stack(batch['input']) # 5 x bz
            y = batch['label'].float() # bz

            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()

            output = model(x)
            diff = output - y
            print("midian:", np.median(np.square(diff)))
            
            loss = criterion(output, y)
            val_loss += loss.item()
#             print(len(dataLoader))
            

    val_loss = val_loss / len(dataLoader)

    del dataLoader

    return val_loss

In [121]:
val(model, test_dataset)

midian: 5469955.0
midian: 5613114.0
midian: 5480575.0
midian: 5666049.0


10379972.75

In [122]:
Token
midian: 518789.12
midian: 514796.2
midian: 532898.94
midian: 513214.62
869457.03125

869457.03125

In [ ]:
Number
midian: 5469955.0
midian: 5613114.0
midian: 5480575.0
midian: 5666049.0
10379972.75